# Build your own pizza MILP


In this notebook, we show the code behind the B\&B process explained in the .pdf tutorial. To recap we are going out for dinner with our friends in a pizzeria that allows customers to build their own pizza. we want to build our own pizza, starting with a pizza margherita that costs 6€. Our budget is 13€. We are given the menu with the list of potential toppings and prices as given below

![image info](./toppings.png)

That defines our set of toppings $\mathcal{T}$, with 12 elements ranging from 1=buffalo mozzarella to 12=bell peppers (see blue numbers in the figure below). In red, we see the satisfaction value that adding such topping generates instead

![image info](./toppings_2.png)



We can now import the needed packages first

In [ ]:
import numpy as np
import os
from gurobipy import Model,GRB,LinExpr
from copy import deepcopy

Now we can define our inputs. We define the set of toppings with the index as the key, and then the full name, price, and satisfaction value

In [2]:
T_names = ["Buffalo mozzarella","Gorgonzola","Ricotta","Burrata","Parma ham","Pancetta",
          "Salame","Nduja","Zucchini","Fried aubergine","Cherry tomatoes","Bell peppers"]
C       = [2,1.5,1,3,3,2,2,0.5,1,2,0.5,1.5]
S       = [5,3.1,4.2,4.7,7.2,4.2,8.3,4.8,3.5,5.2,3.7,4.1]
Budget  = 7

T = {k+1:[T_names[k],C[k],S[k]] for k,_ in enumerate(C)}

In [3]:
T

{1: ['Buffalo mozzarella', 2, 5],
 2: ['Gorgonzola', 1.5, 3.1],
 3: ['Ricotta', 1, 4.2],
 4: ['Burrata', 3, 4.7],
 5: ['Parma ham', 3, 7.2],
 6: ['Pancetta', 2, 4.2],
 7: ['Salame', 2, 8.3],
 8: ['Nduja', 0.5, 4.8],
 9: ['Zucchini', 1, 3.5],
 10: ['Fried aubergine', 2, 5.2],
 11: ['Cherry tomatoes', 0.5, 3.7],
 12: ['Bell peppers', 1.5, 4.1]}

We now manually build the B&B solution tree for this problem. We start with the root node. Note that all the decision variables, which are binary in practice, are relaxed to be continuous

In [4]:
# Setup model
root_node_model = Model()

print('Creating decision variables')
x = {}

for t,_ in T.items():
    # Note: this is the root node, hence all decision variables are relaxed
    # to be continuous
    x[t] = root_node_model.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS,name="x_%s"%(t))
    
print('Creating constraints')
lhs = LinExpr()
for t,_ in T.items():
    lhs += T[t][1]*x[t]
root_node_model.addLConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=Budget,
                    name='Budget')

print('Creating objective function')
obj = LinExpr()
for t,_ in T.items():
    obj += T[t][2]*x[t]


root_node_model.setObjective(obj,GRB.MAXIMIZE)
root_node_model.update()  
  

# Solve
root_node_model.setParam('MIPGap',0.001)
root_node_model.setParam('TimeLimit',2*3600) # seconds
root_node_model.optimize()


solution = []

# Retrieve variable names and values
for v in root_node_model.getVars():
    solution.append([v.varName,v.x])
    
# Retrieve active routing variables
active_variables = []
for i in range(0,len(solution)):
    if solution[i][1] >= 0.01:
        active_variables.append([solution[i][0],solution[i][1]])
        
active_variables

Academic license - for non-commercial use only - expires 2024-08-03
Using license file C:\Users\abombelli\gurobi.lic
Creating decision variables
Creating constraints
Creating objective function
Changed value of parameter MIPGap to 0.001
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 7200.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 12 columns and 12 nonzeros
Model fingerprint: 0xfd3f5062
Coefficient statistics:
  Matrix range     [5e-01, 3e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+00, 7e+00]
Presolve time: 0.00s
Presolved: 1 rows, 12 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.7200000e+01   1.625000e+00   0.000000e+00      0s
       1    2.9900000e+01   0.000000e+00   0.

[['x_3', 1.0],
 ['x_7', 1.0],
 ['x_8', 1.0],
 ['x_9', 1.0],
 ['x_10', 0.25],
 ['x_11', 1.0],
 ['x_12', 1.0]]

We notice that decision variable $x_9$, i.e., zucchini, is fractional. The optimal solution $J=30$ is hence the first best bound $\mathbb{B}\mathbb{B}$ of this problem. We separate on $x_9$, forcing its value to be $x_9=0$ in node 1 and $x_9=1$ in node 2. We can use the property discussed in the tutorial that every child node is a copy of the parent node, plus the additional branching constraint 

In [12]:
node_1_model = root_node_model.copy()
lhs = LinExpr()
lhs += x[10]
node_1_model.addLConstr(lhs=lhs, sense=GRB.EQUAL, rhs=0,
                    name="Branching_constraint_node_1")

node_1_model.update()
node_1_model.optimize()

solution = []

# Retrieve variable names and values
for v in node_1_model.getVars():
    solution.append([v.varName,v.x])

# Retrieve active routing variables
active_variables = []
for i in range(0,len(solution)):
    if solution[i][1] >= 0.01:
        active_variables.append([solution[i][0],solution[i][1]])

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 12 columns and 13 nonzeros
Model fingerprint: 0xf9d0c7ee
Coefficient statistics:
  Matrix range     [5e-01, 3e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+00, 7e+00]
Presolve removed 1 rows and 5 columns
Presolve time: 0.00s
Presolved: 1 rows, 7 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.7200000e+01   3.250000e+00   0.000000e+00      0s
       1    2.9850000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  2.985000000e+01


In [13]:
active_variables

[['x_1', 0.25],
 ['x_3', 1.0],
 ['x_7', 1.0],
 ['x_8', 1.0],
 ['x_9', 1.0],
 ['x_11', 1.0],
 ['x_12', 1.0]]

In [14]:
node_2_model = root_node_model.copy()
lhs = LinExpr()
lhs += x[10]
node_2_model.addLConstr(lhs=lhs, sense=GRB.EQUAL, rhs=1,
                    name="Branching_constraint_node_1")

node_2_model.update()
node_2_model.optimize()

solution = []

# Retrieve variable names and values
for v in node_2_model.getVars():
    solution.append([v.varName,v.x])

# Retrieve active routing variables
active_variables = []
for i in range(0,len(solution)):
    if solution[i][1] >= 0.01:
        active_variables.append([solution[i][0],solution[i][1]])

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 12 columns and 13 nonzeros
Model fingerprint: 0x7e8c47e8
Coefficient statistics:
  Matrix range     [5e-01, 3e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Presolve removed 1 rows and 5 columns
Presolve time: 0.00s
Presolved: 1 rows, 7 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.3200000e+01   2.250000e+00   0.000000e+00      0s
       1    2.9700000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  2.970000000e+01


In [15]:
active_variables

[['x_3', 1.0],
 ['x_7', 1.0],
 ['x_8', 1.0],
 ['x_9', 1.0],
 ['x_10', 1.0],
 ['x_11', 1.0]]

We notice that node 1 still features a fractional solution. On the other hand, node 2 features our first best incumbent $\mathbb{B}\mathbb{I}=29.70$ because all decision variables are non-fractional. Because the root node is now dominated, the new best bound is given by node 1, i.e., $\mathbb{B}\mathbb{B}=29.85$ Note that, apart from $x_{10}$, all other decision variables are still relaxed to be continuous, but they take all non-fractional (1) values in the optimal solution. Our next step is to branch on node 1. The only fractional decision variable is $x_1$, hence there is no ambiguity and we will separate on $x_1$. We use the same strategy of creating a copy of the parent node (node 1, in this case) and of adding the ad-hoc branching constraint to fully define the child node

In [18]:
node_3_model = node_1_model.copy()
lhs = LinExpr()
lhs += x[1]
node_3_model.addLConstr(lhs=lhs, sense=GRB.EQUAL, rhs=0,
                    name="Branching_constraint_node_3")

node_3_model.update()
node_3_model.optimize()

solution = []

# Retrieve variable names and values
for v in node_3_model.getVars():
    solution.append([v.varName,v.x])

# Retrieve active routing variables
active_variables = []
for i in range(0,len(solution)):
    if solution[i][1] >= 0.01:
        active_variables.append([solution[i][0],solution[i][1]])

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3 rows, 12 columns and 14 nonzeros
Model fingerprint: 0xaccdb7a9
Coefficient statistics:
  Matrix range     [5e-01, 3e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+00, 7e+00]
Presolve removed 2 rows and 5 columns
Presolve time: 0.00s
Presolved: 1 rows, 7 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.7200000e+01   1.625000e+00   0.000000e+00      0s
       1    2.9800000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  2.980000000e+01


In [19]:
active_variables

[['x_3', 1.0],
 ['x_5', 0.16666666666666666],
 ['x_7', 1.0],
 ['x_8', 1.0],
 ['x_9', 1.0],
 ['x_11', 1.0],
 ['x_12', 1.0]]

In [20]:
node_4_model = node_1_model.copy()
lhs = LinExpr()
lhs += x[1]
node_4_model.addLConstr(lhs=lhs, sense=GRB.EQUAL, rhs=1,
                    name="Branching_constraint_node_4")

node_4_model.update()
node_4_model.optimize()

solution = []

# Retrieve variable names and values
for v in node_4_model.getVars():
    solution.append([v.varName,v.x])

# Retrieve active routing variables
active_variables = []
for i in range(0,len(solution)):
    if solution[i][1] >= 0.01:
        active_variables.append([solution[i][0],solution[i][1]])

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3 rows, 12 columns and 14 nonzeros
Model fingerprint: 0xd6331df8
Coefficient statistics:
  Matrix range     [5e-01, 3e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Presolve removed 2 rows and 5 columns
Presolve time: 0.01s
Presolved: 1 rows, 7 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.3000000e+01   1.125000e+00   0.000000e+00      0s
       1    2.9500000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  2.950000000e+01


In [21]:
active_variables

[['x_1', 1.0],
 ['x_3', 1.0],
 ['x_7', 1.0],
 ['x_8', 1.0],
 ['x_9', 1.0],
 ['x_11', 1.0]]

We notice that node 3 still features a fractional solution. On the other hand, node 4 features a non-fractional solution where $J=29.50$. Because $J<\mathbb{B}\mathbb{B}$, this solution is dominated by the current best incumbent. Because node 1 is not dominated, the new best bound is $\mathbb{B}\mathbb{B}=29.80$ Our next step is to branch on node 3 and separate the only fractional decision variable $x_5$ in a similar fashion to what done above

In [22]:
node_5_model = node_3_model.copy()
lhs = LinExpr()
lhs += x[5]
node_5_model.addLConstr(lhs=lhs, sense=GRB.EQUAL, rhs=0,
                    name="Branching_constraint_node_5")

node_5_model.update()
node_5_model.optimize()

solution = []

# Retrieve variable names and values
for v in node_5_model.getVars():
    solution.append([v.varName,v.x])

# Retrieve active routing variables
active_variables = []
for i in range(0,len(solution)):
    if solution[i][1] >= 0.01:
        active_variables.append([solution[i][0],solution[i][1]])

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 12 columns and 15 nonzeros
Model fingerprint: 0x91b4771f
Coefficient statistics:
  Matrix range     [5e-01, 3e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+00, 7e+00]
Presolve removed 3 rows and 5 columns
Presolve time: 0.01s
Presolved: 1 rows, 7 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.7200000e+01   3.250000e+00   0.000000e+00      0s
       1    2.9650000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  2.965000000e+01


In [23]:
active_variables

[['x_3', 1.0],
 ['x_6', 0.25],
 ['x_7', 1.0],
 ['x_8', 1.0],
 ['x_9', 1.0],
 ['x_11', 1.0],
 ['x_12', 1.0]]

In [24]:
node_6_model = node_3_model.copy()
lhs = LinExpr()
lhs += x[5]
node_6_model.addLConstr(lhs=lhs, sense=GRB.EQUAL, rhs=1,
                    name="Branching_constraint_node_6")

node_6_model.update()
node_6_model.optimize()

solution = []

# Retrieve variable names and values
for v in node_6_model.getVars():
    solution.append([v.varName,v.x])

# Retrieve active routing variables
active_variables = []
for i in range(0,len(solution)):
    if solution[i][1] >= 0.01:
        active_variables.append([solution[i][0],solution[i][1]])

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 12 columns and 15 nonzeros
Model fingerprint: 0x85f67541
Coefficient statistics:
  Matrix range     [5e-01, 3e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Presolve removed 3 rows and 5 columns
Presolve time: 0.01s
Presolved: 1 rows, 7 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5600000e+01   1.750000e+00   0.000000e+00      0s
       1    2.8200000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  2.820000000e+01


In [25]:
active_variables

[['x_3', 1.0], ['x_5', 1.0], ['x_7', 1.0], ['x_8', 1.0], ['x_11', 1.0]]

We notice that node 5 still features a fractional solution $J=29.65$ which is worse than $\mathbb{B}\mathbb{I}=29.70$. Hence, node 5 is **fathomed**. On the other hand, node 6 features a non-fractional solution where $J=29.82$. Because $J<\mathbb{B}\mathbb{B}$, this solution is dominated by the current best incumbent. We have no more nodes with that are unexplored, hence our search is over. We artificially lower our $\mathbb{B}\mathbb{B}$ to $29.70$ to match $\mathbb{B}\mathbb{I}$, and our optimal solution is $J=29.70$ where $x_3=x_7=x_8=x_9=x_{10}=x_{11}=1$.   

To confirm this, we now run the MILP where decision variables are binary, and let the solver carry out the B\&B process for us

In [27]:
# Setup model
model = Model()

print('Creating decision variables')
x = {}

for t,_ in T.items():
    x[t] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY,name="x_%s"%(t))
    
print('Creating constraints')
lhs = LinExpr()
for t,_ in T.items():
    lhs += T[t][1]*x[t]
model.addLConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=Budget,
                    name='Budget')

print('Creating objective function')
obj = LinExpr()
for t,_ in T.items():
    obj += T[t][2]*x[t]


model.setObjective(obj,GRB.MAXIMIZE)
model.update()  
  

# Solve
model.setParam('MIPGap',0.001)
model.setParam('TimeLimit',2*3600) # seconds
model.optimize()


solution = []

# Retrieve variable names and values
for v in model.getVars():
    solution.append([v.varName,v.x])
    
# Retrieve active routing variables
active_variables = []
for i in range(0,len(solution)):
    if solution[i][1] >= 0.01:
        active_variables.append([solution[i][0],solution[i][1]])
        
active_variables

Creating decision variables
Creating constraints
Creating objective function
Changed value of parameter MIPGap to 0.001
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 7200.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 12 columns and 12 nonzeros
Model fingerprint: 0x76f1b785
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [5e-01, 3e+00]
  Objective range  [3e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+00, 7e+00]
Found heuristic solution: objective 21.3000000
Presolve time: 0.01s
Presolved: 1 rows, 12 columns, 12 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)

Root relaxation: objective 2.990000e+01, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     

[['x_3', 1.0],
 ['x_7', 1.0],
 ['x_8', 1.0],
 ['x_9', 1.0],
 ['x_10', 1.0],
 ['x_11', 1.0]]

We notice that the optimal solution is consistent with what we found with our home-made B\&B. It should also be noted that Gurobi only explored a single node (the **root node**), while we had to explore 7. The reason behind this is that Gurobi employs a lot of pre-processing techniques that simplify the problem already, and is geenrally capable of eliminating some decision variables/constraints even before starting the B\&B procedure. What is important here is that, for this simple example, we were capable of fully exploring the B\&B decision tree and show how the optimal solution is obtained by keeping trach of the best bound $\mathbb{B}\mathbb{B}$, the best incumbent $\mathbb{B}\mathbb{I}$, and applying fathoming rules.